In [14]:
# from numba import(jitclass, njit, prange, types)
from numba import(jit)
import numpy as np
import numpy.random as nr
from scipy.linalg import(expm)
import time
from joblib import(Parallel, delayed)
import multiprocessing

In [15]:
from compression import(compress_circuit)
from frame_opt import(init_x_list, get_negativity_circuit, sequential_para_opt)
from phase_space import(PhaseSpace)
from qubit_frame_Wigner import(F, G, DIM, x0)

In [16]:
def get_qd_output(circuit, par_list, ps):
    '''
    Calculate the quasi-probability distribution of each circuit element
    and return them as a list of [state qds, gate qds, measurement qds].
    '''
    par_idx_states = np.arange(0, len(circuit["state_list"]))
    par_idx_gates = par_list[1]
    par_idx_meas = par_list[2]
    par_vals = par_list[0]

    qd_list_states = []   # qd lists
    qd_list_gates = []
    qd_list_meas = []

    neg_list_states = []  # qd negativity lists
    neg_list_gates = []
    neg_list_meas = []

    sign_list_states = [] # sign of qd lists
    sign_list_gates = []
    sign_list_meas = []

    pd_list_states = []   # normalised prob dist lists
    pd_list_gates = []
    pd_list_meas = []

    # States
    for s, state in enumerate(circuit['state_list']):
        x = par_vals[par_idx_states[s]]

        qd_state = ps.W_state(state, x)
        pd_state = np.abs(qd_state)
        neg_state = pd_state.sum()
        pd_state /= neg_state
        sign_state = np.sign(qd_state)

        qd_list_states.append(qd_state)
        pd_list_states.append(pd_state)
        neg_list_states.append(neg_state)
        sign_list_states.append(sign_state)

    # Gates
    for g, gate in enumerate(circuit['gate_list']):
        idx = circuit['index_list'][g]

        x_in, x_out = [], []
        for k in par_idx_gates[g][0]:
            x_in.append(par_vals[k])
        for k in par_idx_gates[g][1]:
            x_out.append(par_vals[k])

        qd_gate = ps.W_gate(gate, x_in, x_out)
        pd_gate = np.abs(qd_gate)
        neg_gate = pd_gate.sum(axis=tuple(range(2*len(idx),4*len(idx))))
        for i in range(2*len(idx)):
            pd_gate = pd_gate.swapaxes(i,2*len(idx)+i)
        pd_gate = pd_gate/neg_gate
        for i in range(2*len(idx)):
            pd_gate = pd_gate.swapaxes(i,2*len(idx)+i)
        sign_gate = np.sign(qd_gate)

        qd_list_gates.append(qd_gate)
        pd_list_gates.append(pd_gate)
        neg_list_gates.append(neg_gate)
        sign_list_gates.append(sign_gate)

    # Measurements
    for m, meas in enumerate(circuit['meas_list']):
        x = par_vals[par_idx_meas[m]]

        qd_meas = ps.W_meas(meas, x)
        pd_meas = np.abs(qd_meas)
        neg_meas = np.max(pd_meas)
        sign_meas = np.sign(qd_meas)

        qd_list_meas.append(qd_meas)
        pd_list_meas.append(pd_meas)
        neg_list_meas.append(neg_meas)
        sign_list_meas.append(sign_meas)

    output = {
    'qd_list_states': qd_list_states, 'qd_list_gates': qd_list_gates,
    'qd_list_meas': qd_list_meas, 'pd_list_states': pd_list_states,
    'pd_list_gates': pd_list_gates, 'pd_list_meas': pd_list_meas,
    'neg_list_states': neg_list_states, 'neg_list_gates': neg_list_gates,
    'neg_list_meas': neg_list_meas, 'sign_list_states': sign_list_states,
    'sign_list_gates': sign_list_gates, 'sign_list_meas': sign_list_meas
    }
    return output

In [17]:
def prepare_sampler(circuit, par_list, ps):
    meas_list = np.stack(circuit["meas_list"]).astype(np.float32)
    index_list = np.array(circuit["index_list"]).astype(np.int8)

    output = get_qd_output(circuit, par_list, ps)

    qd_list_states = np.stack([dist.flatten().astype(np.float32)
                               for dist in output["qd_list_states"]])
    qd_list_gates = np.stack([dist.flatten().astype(np.float32)
                              for dist in output["qd_list_gates"]])
    qd_list_meas = np.stack([dist.flatten().astype(np.float32)
                             for dist in output["qd_list_meas"]])

    neg_list_states = np.array(output["neg_list_states"]).astype(np.float32)
    neg_list_gates = np.stack([dist.flatten().astype(np.float32)
                               for dist in output["neg_list_gates"]])
    neg_list_meas = np.array(output["neg_list_meas"]).astype(np.float32)

    pd_list_states = np.stack([dist.flatten().astype(np.float32)
                               for dist in output["pd_list_states"]])
    pd_list_gates = np.stack([dist.flatten().astype(np.float32)
                              for dist in output["pd_list_gates"]])
    pd_list_meas = np.stack([dist.flatten().astype(np.float32)
                             for dist in output["pd_list_meas"]])

    sign_list_states = np.stack([dist.flatten().astype(np.float32)
                                 for dist in output["sign_list_states"]])
    sign_list_gates = np.stack([dist.flatten().astype(np.float32)
                                for dist in output["sign_list_gates"]])
    sign_list_meas = np.stack([dist.flatten().astype(np.float32)
                               for dist in output["sign_list_meas"]])

    return(meas_list, index_list, qd_list_states, qd_list_gates, qd_list_meas,
           pd_list_states, pd_list_gates, pd_list_meas, sign_list_states,
           sign_list_gates, sign_list_meas, neg_list_states, neg_list_gates,
           neg_list_meas)

In [29]:
#@jit(nopython=True)
def choice(prob_dist):
    return(np.arange(len(prob_dist))[np.searchsorted(
             np.cumsum(prob_dist), np.random.random(), side="right")])

def sample_fast(meas_list, index_list, qd_list_states, qd_list_gates, 
                qd_list_meas, pd_list_states, pd_list_gates, pd_list_meas, 
                sign_list_states, sign_list_gates, sign_list_meas, 
                neg_list_states, neg_list_gates, neg_list_meas):
    N = meas_list.shape[0]
    current_ps_point = np.zeros(N, dtype=np.int32)
    p_estimate = 1.

    # Input states
    for s in range(N):
        ps_point = choice(pd_list_states[s])
        current_ps_point[s] = ps_point
        p_estimate *= neg_list_states[s]*sign_list_states[s][ps_point]

    # Gates
    for g in range(index_list.shape[0]):
        idx = index_list[g]

        arr_dim = np.log2(len(pd_list_gates[g]))/2
        pq_in = 0
        for i in range(len(idx)):
            pq_in += current_ps_point[idx[i]]//2 * 2**(2*(arr_dim-i)-1)
            pq_in += current_ps_point[idx[i]]%2 * 2**(2*(arr_dim-i)-2)
        pq_in = int(pq_in)

        prob = pd_list_gates[g,pq_in:pq_in+int(2**arr_dim)]
        sign = sign_list_gates[g,pq_in:pq_in+int(2**arr_dim)]
        neg = neg_list_gates[g,int(pq_in//(2**arr_dim))]

        ps_point = choice(prob)
        prob_dim = np.log2(len(prob))/2
        for i in range(len(idx)):
            current_ps_point[idx[i]] = int(ps_point/4**(prob_dim-1-i))%4
        p_estimate *= neg*sign[ps_point]


    # Measurement
    exp_qaoa = p_estimate
    temp = 0
    for m in range(N):
        p_estimate *= qd_list_meas[m,current_ps_point[m]]

        m_next = (m+1)%N
        temp += 0.5 * qd_list_meas[m_next,current_ps_point[m_next]]*\
                      qd_list_meas[m,current_ps_point[m]]
    exp_qaoa *= temp
    
    return exp_qaoa # p_estimate # 

In [19]:
def sample(sample_size, circuit, par_list, ps):
    
    meas_list, index_list, qd_list_states, qd_list_gates, qd_list_meas,\
    pd_list_states, pd_list_gates, pd_list_meas, sign_list_states,\
    sign_list_gates, sign_list_meas, neg_list_states, neg_list_gates,\
    neg_list_meas = prepare_sampler(circuit, par_list, ps)
    
    p_est = np.zeros(sample_size)
    for i in range(sample_size):
        if i%(sample_size//10)==0: print(i/sample_size*100, "%")
        p_est[i] = sample_fast(meas_list, index_list, qd_list_states, 
                     qd_list_gates,  qd_list_meas, pd_list_states, 
                     pd_list_gates, pd_list_meas,  sign_list_states, 
                     sign_list_gates, sign_list_meas,  neg_list_states, 
                     neg_list_gates, neg_list_meas)
    return 1./sample_size * sum(p_est)

def sample_parallel(sample_size, circuit, par_list, ps):
    
    meas_list, index_list, qd_list_states, qd_list_gates, qd_list_meas,\
    pd_list_states, pd_list_gates, pd_list_meas, sign_list_states,\
    sign_list_gates, sign_list_meas, neg_list_states, neg_list_gates,\
    neg_list_meas = prepare_sampler(circuit, par_list, ps)
    
    def sample_func(sample_size):
        tally = 0.
        for i in range(sample_size):
            tally += sample_fast(meas_list, index_list, qd_list_states, qd_list_gates, 
                qd_list_meas, pd_list_states, pd_list_gates, pd_list_meas, 
                sign_list_states, sign_list_gates, sign_list_meas, 
                neg_list_states, neg_list_gates, neg_list_meas)
        return tally
    
    n_core = multiprocessing.cpu_count()
    p_est = Parallel(n_jobs=n_core)(delayed(sample_func)(sample_size//n_core) for i in range(n_core))
    return 1./sample_size * sum(p_est)

In [20]:
ps_Wigner = PhaseSpace(F, G, x0, DIM)

plus = 0.5*np.array([[1.,1],[1.,1]])
Z    = np.array([[1,0],[0,-1]])
ZZ   = np.kron(Z,Z)
X    = np.array([[0,1],[1,0]])

def U_prob(gamma):
    return expm(-1.j*2*gamma*ZZ)
def U_mix(beta):
    return expm(-1.j*beta*X)

class Graph():
    def __init__(self, N, edges):
        self.N = N
        self.nodes = np.arange(N)
        self.edges = [list(edge) for edge in edges]

def qaoa_maxcut(G, beta_list, gamma_list):
    state_list = G.N*[plus]

    gate_list = []
    index_list = []
    for i, beta in enumerate(beta_list):
        gamma = gamma_list[i]
        UP = U_prob(gamma)
        for edge in G.edges:
            gate_list.append(UP)
            index_list.append(edge)
        UM = U_mix(beta)
        for node in range(G.N):
            gate_list.append(UM)
            index_list.append([node])

    meas_list = G.N*[Z]

    circuit = {'state_list': state_list, 'gate_list': gate_list,
               'index_list': index_list, 'meas_list': meas_list}
    return circuit

x0 = ps_Wigner.x0
W = ps_Wigner.W

edges = [(0, 1), (1, 2), (2, 3), (3, 0)]
G = Graph(4, edges)

In [21]:
beta, gamma = 0.1, 0.2
circuit = qaoa_maxcut(G, [beta], [gamma])
circuit = compress_circuit(circuit, n=2)
x_circ  = init_x_list(circuit, x0)
x_out, neg_list = sequential_para_opt(W, circuit, x_circ, l=1, niter=1)

In [28]:
sample_size = int(1e6)
print("STEP 0")
t0 = time.time()
out1 = sample(sample_size, circuit=circuit, par_list=x_out, ps=ps_Wigner)
t1 = time.time() - t0
# print("STEP 1")
# t0 = time.time()
# out2 = sample_parallel(sample_size, circuit=circuit, par_list=x_out, ps=ps_Wigner)
# t2 = time.time() - t0
# print("STEP 2")

STEP 0
0.0 %
10.0 %
20.0 %
30.0 %


KeyboardInterrupt: 

In [26]:
t1

29.561963081359863

In [43]:
t2

13.714343070983887

In [44]:
out1

0.12767408328967741

In [45]:
out2

0.12603131562570194